# Score 계산 수식

## 1. Normalized RMSE 계산
- 수식:
    $$
    A = \text{Normalized RMSE} = \frac{\text{RMSE}}{\max(\text{IC50}_{\text{true}}) - \min(\text{IC50}_{\text{true}})}
    $$

- RMSE 계산:
    $$
    \text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (\text{predicted\_values}_i - \text{true\_values}_i)^2}
    $$

- Normalized RMSE 계산:
    $$
    \text{Normalized RMSE} = \frac{\text{RMSE}}{\max(\text{true\_values}) - \min(\text{true\_values})}
    $$

## 2. Correct Ratio (Absolute Error <= 0.5) 계산
- 수식:
    $$
    B = \text{Correct Ratio} = \frac{1}{N} \sum_{i=1}^{N} I\left(\left| \text{Absolute Error of } pIC50_i \right| \leq 0.5\right)
    $$

- 절대 오차 계산:
    $$
    \text{Absolute Error} = \left| \text{predicted\_pIC50}_i - \text{true\_pIC50}_i \right|
    $$

- Correct Ratio 계산:
    $$
    \text{Correct Ratio} = \frac{1}{N} \sum_{i=1}^{N} I\left(\left| \text{predicted\_pIC50}_i - \text{true\_pIC50}_i \right| \leq 0.5\right)
    $$

    여기서 $I$는 조건을 만족할 때 1을, 만족하지 않을 때 0을 반환하는 지시 함수입니다.

## 3. 최종 Score 계산
- 수식:
    $$
    \text{Score} = 0.5 \times (1 - \min(A, 1)) + 0.5 \times B
    $$

- 최종 Score 계산:
    $$
    \text{Score} = 0.5 \times (1 - \min(\text{Normalized RMSE}, 1)) + 0.5 \times \text{Correct Ratio}
    $$

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np


# pIC50에서 IC50을 계산하는 함수
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

# Normalized RMSE를 계산하는 함수
def calculate_normalized_rmse(true_pIC50, predicted_pIC50):
    """
    Normalized RMSE 계산.
    Normalized RMSE = RMSE / (max(true_IC50) - min(true_IC50))
    """
    # RMSE 계산
    mse = mean_squared_error(pIC50_to_IC50(true_pIC50), pIC50_to_IC50(predicted_pIC50))
    rmse = np.sqrt(mse)
    # Normalized RMSE 계산
    norm_rmse = rmse / (np.max(pIC50_to_IC50(true_pIC50)) - np.min(pIC50_to_IC50(predicted_pIC50)))
    return norm_rmse

# pIC50에서 Correct Ratio를 계산하는 함수
def calculate_correct_ratio(true_pIC50, predicted_pIC50):
    """
    Correct Ratio 계산.
    Correct Ratio = 1/N * Σ I(Absolute Error <= 0.5)
    """
    # 절대 오차 계산
    abs_errors = np.abs(predicted_pIC50 - true_pIC50)
    # 절대 오차가 0.5 이하인 경우의 수 계산
    correct_count = np.sum(abs_errors <= 0.5)
    # Correct Ratio 계산
    correct_ratio = correct_count / len(true_pIC50)
    return correct_ratio

# 최종 Score를 계산하는 함수
def calculate_score(true_pIC50, predicted_pIC50):
    """
    주어진 실제 IC50 값과 예측 IC50 값을 기반으로 Score 계산.
    Score = 0.5 * (1 - min(Normalized RMSE, 1)) + 0.5 * Correct Ratio
    """
    # Normalized RMSE 계산 (A 값)
    norm_rmse = calculate_normalized_rmse(true_pIC50, predicted_pIC50)

    # Correct Ratio 계산 (B 값)
    correct_ratio = calculate_correct_ratio(true_pIC50, predicted_pIC50)

    # 최종 Score 계산
    # Score = 0.5 * (1 - min(Normalized RMSE, 1)) + 0.5 * Correct Ratio
    score = 0.5 * (1 - min(norm_rmse, 1)) + 0.5 * correct_ratio
    
    return score

